In [1]:
import os
import requests
import json
import csv
import re
from time import sleep

In [ ]:
class stockDataDownload(object):
    def __init__(self):
        # 当前文件所处目录
        self.path = os.path.abspath(os.getcwd())
        # 所有股票代码API
        self.tick_code_api = 'https://api.doctorxiong.club/v1/stock/all'
        # 股票历史交易数据API（网易财经）
        self.tick_his_api = 'http://quotes.money.163.com/service/chddata.html'


    def stock_code_save(self, dirname):
        try:
            if not os.path.exists(dirname):
                print ('文件夹', dirname, '不存在，已重新建立')
                os.makedirs(dirname)

            # 所有股票代码数据存储文件名称
            all_stock_code = 'all_stock_code.csv'
            filepath = os.path.join(dirname, all_stock_code)

            html = requests.get(self.tick_code_api).text
            content = json.loads(html)

            with open (filepath, 'w') as w:
                w.write('StockCode,StockName\n')
                for i in content['data']:
                    t = i[0] + ',' + i[1] + '\n'
                    w.write(t)
            w.close()
            print('已完成所有股票代码下载')

        except IOError as e:
            print ('文件操作失败',e)
        except Exception as e:
            print ('错误 ：',e)

    def tick_his_download(self, stickcode, dirname):
        '''
        下载指定代码股票从1990.01.01开始的日K数据
        接口采用网易财经接口：'http://quotes.money.163.com/service/chddata.html?code=0601012&start=20120411&end=20210908&fields=TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;TURNOVER;VOTURNOVER;VATURNOVER;TCAP;MCAP'
        code: 股票代码
        start/end：开始日期/结束日期
        fields:具体下载的字段
            TCLOSE：当日收盘价
            HIGH：当日最高价
            LOW：当日最低价
            TOPEN：当日开盘价
            LCLOSE：昨日收盘价
            CHG：涨跌额
            PCHG：涨跌幅
            TURNOVER：换手率
            VOTURNOVER：成交量
            VATURNOVER：成交额
            TCAP：总市值
            MCAP：流通市值
        '''
        try:
            print(os.getcwd())
            if not os.path.exists(dirname):
                print ('文件夹', dirname, '不存在，已重新建立')
                os.makedirs(dirname)

            # 下载股票数据
            param = {
                'code': stickcode,
                'start': '19900101',
                'fields': 'TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;TURNOVER;VOTURNOVER;VATURNOVER;TCAP;MCAP'
            }
            print('开始下载股票', stickcode, '...')
            response = requests.get(self.tick_his_api, params=param)
            if response.status_code!=200:
                print('股票', stickcode, '下载失败，开始下一个任务...')
                return
            filepath = os.path.join(dirname, stickcode+'.csv')
            with open(filepath, mode='wb') as w:
                w.write(response.text.encode('utf8'))
            w.close()
            print('已正常存储', filepath)

        except IOError as e:
            print ('文件操作失败',e)
        except Exception as e:
            print ('错误 ：',e)

    # 获取当前路径，文件夹，文件
    def get_file_info(self, file_dir):
        for root, dirs, files in os.walk(file_dir):
            # print(root)
            # print(dirs)
            # print(files)
            pass
        return root, dirs, files

    # 删除路径下的小文件（文件大小小于0.17k）
    def remove_blank_csv(self, path):
        for root, dirs, files in os.walk(path):
            pass
        for file in files:
            filepath = os.path.join(root, file)
            if os.path.getsize(filepath)/1024 < 0.17:
                os.remove(filepath)



In [ ]:
# 调试
# 检测已下载的跳过，继续下载后续数据

sdd = stockDataDownload()
# sdd.stock_code_save(dirname='dataDownload')

# 切换到数据存储文件夹，读取所有股票代码数据
os.chdir(os.path.join(os.path.abspath(os.path.dirname(os.getcwd())), 'dataDownload'))
print(os.getcwd())
# 提出已下载数据内的空文件
sdd.remove_blank_csv(os.getcwd())
stockCodefile = os.path.join(os.getcwd(), 'all_stock_code.csv')
# 获取已下载的数据
root, dirs, files = sdd.get_file_info(os.getcwd())


# 切换到项目根目录
os.chdir(os.path.dirname(os.getcwd()))
with open(stockCodefile, mode='r') as r:
    f_csv = csv.reader(r)
    for row in f_csv:
        if row[0]=='StockCode' or '0'+row[0][2:]+'.csv' in files:
            print('0'+row[0][2:]+'.csv', '已存在')
            continue
        print('当前目标股票：', row)
        sdd.tick_his_download(stickcode='0'+row[0][2:], dirname='dataDownload')
        sleep(2)
r.close()

D:\个人\Python学习\AutoStock\DataCollect


In [ ]:
sdd = stockDataDownload()
# os.chdir(os.path.join(os.path.abspath(os.path.dirname(os.getcwd())), 'dataDownload'))
print(os.getcwd())
root, dirs, files = sdd.get_file_info(os.getcwd())
print(root, dirs, files)
'''size1 = os.path.getsize(os.path.join(root, '0000760.csv'))/1024  # Kb
size2 = os.path.getsize(os.path.join(root, '0688187.csv'))/1024  # Kb
print(size1,size2)'''
'''# 删除空文件
for file in files:
    filepath = os.path.join(root, file)
    if os.path.getsize(filepath)/1024 < 0.17:
        os.remove(filepath)'''
        

c:\Users\Chenc\OneDrive - whut.edu.cn\个人\AutoStock\DataCollect
c:\Users\Chenc\OneDrive - whut.edu.cn\个人\AutoStock\DataCollect\.ipynb_checkpoints [] ['Untitled-checkpoint.ipynb']


'# 删除空文件\nfor file in files:\n    filepath = os.path.join(root, file)\n    if os.path.getsize(filepath)/1024 < 0.17:\n        os.remove(filepath)'

In [2]:
# 获取除权数据

url = 'http://quotes.money.163.com/f10/fhpg_600066.html'
html = requests.get(url=url).text

In [ ]:
url2 = 'http://quotes.money.163.com/f10/fhpg_688819.html'
html2 = requests.get(url2).text
print(html2)

In [8]:
# print(html2)
fhpg_pattern = '分红年度(.*?)<span class="name">配股一览</span>'
pgyl_pattern = '<span class="name">配股一览</span>(.*?)<span class="name">增发一览</span>'
zfyl_pattern = '<span class="name">增发一览</span>(.*?)<span class="name">历年融资计划</span>'

fhpg_content = re.findall(pattern=fhpg_pattern,string=html2,flags=re.S)[0]
pgyl_content = re.findall(pattern=pgyl_pattern, string=html2, flags=re.S)[0]
zfyl_content = re.findall(pattern=zfyl_pattern, string=html2, flags=re.S)[0]

fhpg_detail_pattern = "<tr class=.*?'align_c'>(.*?)</td><td class='align_c'>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td class='align_c'>(.*?)</td><td class='align_c'>(.*?)</td><td class='align_c'>(.*?)</td></tr>"
pgyl_detail_pattern = "<tr class=.*?'align_c'>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td class='align_c'>(.*?)</td></tr>"
zfyl_detail_pattern = "<tr class=.*?'align_c'>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td class='breakline'>(.*?)</td><td class='align_c'>(.*?)</td><td class='align_c'>(.*?)</td></tr>"

fhpg_details = re.findall(pattern=fhpg_detail_pattern, string=fhpg_content)
pgyl_details = re.findall(pattern=pgyl_detail_pattern, string=pgyl_content)
zfyl_details = re.findall(pattern=zfyl_detail_pattern, string=zfyl_content)


In [9]:
print(fhpg_details)
print(pgyl_details)
print(zfyl_details)


[('2021-05-15', '2020', '--', '--', '6.00', '2021-05-20', '2021-05-21', '--')]
[]
[]


In [48]:
# 访问网易财经的url，获取相关HTML信息，解析后存储为json文件
def Get_fhpg_info(tick_code, delay_time):
    # tick_code: 股票编码
    # delay_time: 延时时间
    # data_save_path: 分红信息存储文件绝对路径
    # URL: http://quotes.money.163.com/f10/fhpg_ + tick_code + .html
    sleep(1)

    url = 'http://quotes.money.163.com/f10/fhpg_'+tick_code+'.html'
    html = requests.get(url).text

    fhpg_pattern = '分红年度(.*?)<span class="name">配股一览</span>'
    pgyl_pattern = '<span class="name">配股一览</span>(.*?)<span class="name">增发一览</span>'
    zfyl_pattern = '<span class="name">增发一览</span>(.*?)<span class="name">历年融资计划</span>'

    fhpg_content = re.findall(pattern=fhpg_pattern,string=html,flags=re.S)[0]
    pgyl_content = re.findall(pattern=pgyl_pattern, string=html, flags=re.S)[0]
    zfyl_content = re.findall(pattern=zfyl_pattern, string=html, flags=re.S)[0]

    fhpg_detail_pattern = "<tr class=.*?'align_c'>(.*?)</td><td class='align_c'>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td class='align_c'>(.*?)</td><td class='align_c'>(.*?)</td><td class='align_c'>(.*?)</td></tr>"
    pgyl_detail_pattern = "<tr class=.*?'align_c'>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td class='align_c'>(.*?)</td></tr>"
    zfyl_detail_pattern = "<tr class=.*?'align_c'>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td>(.*?)</td><td class='breakline'>(.*?)</td><td class='align_c'>(.*?)</td><td class='align_c'>(.*?)</td></tr>"

    fhpg_details = re.findall(pattern=fhpg_detail_pattern, string=fhpg_content)
    pgyl_details = re.findall(pattern=pgyl_detail_pattern, string=pgyl_content)
    zfyl_details = re.findall(pattern=zfyl_detail_pattern, string=zfyl_content)


    # 分红配股的Key值
    fhpg_keys = ['公告日期', '分红年度', '10股送股', '10股转增', '10股派息', '股权登记日', '红股上市日']
    # 配股一览的Key值
    pgyl_keys = ['公告日期', '配股方案', '配股价', '配股对象', '实际配股数', '配股前总股本', '实际配股比例', '配股上市日']
    # 增发一览的Key值
    zfyl_keys = ['公告日期', '融资类型', '发行数量（万股）', '募资总额（万元）', '发行价格', '发行方式', '股权登记日', '新股上市日']

    fhpg_info_list = []
    pgyl_info_list = []
    zfyl_info_list = []

    for info in fhpg_details:
        fhpg_info = dict(zip(fhpg_keys, info))
        fhpg_info_list.append(fhpg_info)
    
    for info in pgyl_details:
        pgyl_info = dict(zip(pgyl_keys, info))
        pgyl_info_list.append(pgyl_info)

    for info in zfyl_details:
        zfyl_info = dict(zip(zfyl_keys, info))
        zfyl_info_list.append(zfyl_info)
    
    fh_info_keys = ['分红配股', '配股一览', '增发一览']
    fh_info = dict(zip(fh_info_keys, (fhpg_info_list, pgyl_info_list, zfyl_info_list)))

    return fh_info
    

    

In [ ]:
param = {
    'code': '000024',
    'start': '19900101',
    'fields': 'TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;TURNOVER;VOTURNOVER;VATURNOVER;TCAP;MCAP'
}
response = requests.get('http://quotes.money.163.com/service/chddata.html', params=param)

In [49]:
# info = Get_fhpg_info(tick_code='000024', delay_time=10, data_save_path='1')
info2 = Get_fhpg_info(tick_code='600066', delay_time=10, data_save_path='1')

[{'公告日期': '2021-05-17', '分红年度': '2020', '10股送股': '--', '10股转增': '--', '10股派息': '5.00', '股权登记日': '2021-05-20', '红股上市日': '2021-05-21'}, {'公告日期': '2020-05-14', '分红年度': '2019', '10股送股': '--', '10股转增': '--', '10股派息': '10.00', '股权登记日': '2020-05-19', '红股上市日': '2020-05-20'}, {'公告日期': '2019-05-09', '分红年度': '2018', '10股送股': '--', '10股转增': '--', '10股派息': '5.00', '股权登记日': '2019-05-14', '红股上市日': '2019-05-15'}, {'公告日期': '2018-07-20', '分红年度': '2017', '10股送股': '--', '10股转增': '--', '10股派息': '5.00', '股权登记日': '2018-07-26', '红股上市日': '2018-07-27'}, {'公告日期': '2017-06-06', '分红年度': '2016', '10股送股': '--', '10股转增': '--', '10股派息': '10.00', '股权登记日': '2017-06-13', '红股上市日': '2017-06-14'}, {'公告日期': '2016-05-13', '分红年度': '2015', '10股送股': '--', '10股转增': '--', '10股派息': '15.00', '股权登记日': '2016-05-20', '红股上市日': '2016-05-23'}, {'公告日期': '2015-05-14', '分红年度': '2014', '10股送股': '0', '10股转增': '5', '10股派息': '10.00', '股权登记日': '2015-05-19', '红股上市日': '2015-05-20'}, {'公告日期': '2014-05-20', '分红年度': '2013', '10股送股': '--', '10股转增': '--

In [50]:
print(info2)

{'分红配股': [{'公告日期': '2021-05-17', '分红年度': '2020', '10股送股': '--', '10股转增': '--', '10股派息': '5.00', '股权登记日': '2021-05-20', '红股上市日': '2021-05-21'}, {'公告日期': '2020-05-14', '分红年度': '2019', '10股送股': '--', '10股转增': '--', '10股派息': '10.00', '股权登记日': '2020-05-19', '红股上市日': '2020-05-20'}, {'公告日期': '2019-05-09', '分红年度': '2018', '10股送股': '--', '10股转增': '--', '10股派息': '5.00', '股权登记日': '2019-05-14', '红股上市日': '2019-05-15'}, {'公告日期': '2018-07-20', '分红年度': '2017', '10股送股': '--', '10股转增': '--', '10股派息': '5.00', '股权登记日': '2018-07-26', '红股上市日': '2018-07-27'}, {'公告日期': '2017-06-06', '分红年度': '2016', '10股送股': '--', '10股转增': '--', '10股派息': '10.00', '股权登记日': '2017-06-13', '红股上市日': '2017-06-14'}, {'公告日期': '2016-05-13', '分红年度': '2015', '10股送股': '--', '10股转增': '--', '10股派息': '15.00', '股权登记日': '2016-05-20', '红股上市日': '2016-05-23'}, {'公告日期': '2015-05-14', '分红年度': '2014', '10股送股': '0', '10股转增': '5', '10股派息': '10.00', '股权登记日': '2015-05-19', '红股上市日': '2015-05-20'}, {'公告日期': '2014-05-20', '分红年度': '2013', '10股送股': '--', '10

In [17]:
dic1 = {'a':{'m':1,'n':2}, 'b':{'x':2}}
print(dic1['a'])
print(dic1['a']['m'])
json.dumps(dic1)

{'m': 1, 'n': 2}
1


'{"a": {"m": 1, "n": 2}, "b": {"x": 2}}'

In [40]:
la = [1,2,3]
lb = []
lc = []
ll = dict(zip(['a','b','c'],(la, lb, lc)))
print(ll)

{'a': [1, 2, 3], 'b': [], 'c': []}
